# Detekce defektů pomocí Fourierovy transformace
Na dnešním cvičení bude vaším úkolem vytvořit algoritmus na kontrolu kvality kartonu. Algoritmus by měl být schopný detekovat trhliny, díry a jiné defekty. Jelikož kontrolovaný karton má opakující se strukturu, bude náš algoritmus využívat Fourierovy transformace. Na našem konkrétním kartonu se defekty vyskytují jen v podobě promáčklin na některých vlnách, čímž se narušil jejich pravidelný vzor. Vaším úkolem bude tyto defekty za pomoci Fourierovy transformace v obrázku najít a vyznačit.

Obrázek kartonu byl nasnímán řádkovou kamerou. Kamera byla statická, karton se pod ní pohyboval pomocí důmyslného dopravníku.

## Řádkové kamery
Řádková kamera neboli řádkový skener je speciální druh kamery, která snímá pouze 1 řádek. Díky tomu je schopna dosáhnout obrovských frekvencí snímání (až ticíce řádků za 1 sekundu). V labu máme kamery [Basler Racer](https://www.baslerweb.com/en/products/cameras/line-scan-cameras/racer/) o velikosti řádku 6k, 8k a 12k pixelů. Řádkové kamery jsou díky svým specifickým vlastnostem vynikající pro řadu úloh, typickým použitím je snímání nekonečného pásu ve výrobních linkách.

<img src="images/racer.png" width="25%">

Z podstaty skeneru však vyplývá, že se objekt pod kamerou musí pohybovat. V případě, že se objekt nebude pohybovat, vytvoří kamera obrázek, který bude mít všechny řádky totožné (pouze ten 1, který kamera snímá).

Tím, že je kamera schopná dosahovat neskutečných frekvencí snímání dále umožňuje získávat obrazová data v neskutečně velkém rozlišení. Na druhou stranu nás tím nutí nastavovat nízkou dobu expozice a tedy je zapotřebí mnohem více světla.

### Import knihoven a konfigurace

In [ ]:
%run ../../svz.ipynb

### Pomocné funkce

Z funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

- [`show_images(...)`](../svz.ipynb#show_functions)
- [`plot_images(...)`](../svz.ipynb#show_functions)
- [`load_image(...)`](../svz.ipynb#load_save_functions)


- [`crop(...)`](../svz.ipynb#preprocessing_functions)
- [`filtration_median(...)`](../svz.ipynb#preprocessing_functions)
- [`segmentation_one_threshold(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_auto_threshold(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_two_thresholds(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_adaptive_threshold(...)`](../svz.ipynb#segmentation_functions)

Následující funkce jsou nové a umožňují nám provést Fourierovu transformaci a související operace.

In [ ]:
def apply_fft ( image ):
    ''' Applies FFT on image given.
    
    Parameters
    ----------
    image : 2D array
        Image to perform FFT on.
    Returns
    -------
    mag_spec : 2D array
        Normalized magnitude spectrum.
    fftcls_shift : 2D array
        Centered product of FFT.
    '''
    fftcls = np.fft.fft2(image)
    fftcls_shift = np.fft.fftshift(fftcls)
    mag_spec = 20*np.log(np.abs(fftcls_shift))
    return cv2.normalize(mag_spec,  None, 0, 255, cv2.NORM_MINMAX,cv2.CV_8U), fftcls_shift

In [ ]:
def inverse_fft (fft_shift, filter_mask):
    ''' Applies inverse FFT.
    
    Parameters
    ----------
    fft_shift : 2D array
        Shifted computed FFT
    filter_mask : 2D array
        2D array mask containing 255 and 0 values.
    Returns
    -------
    img_back : 2D array
        Image made by inverse FFT.
    '''
    fftshift = np.copy(fft_shift)
    fftshift[filter_mask != 255] = 0

    f_ishift = np.fft.ifftshift(fftshift)
    return np.abs(np.fft.ifft2(f_ishift))

In [ ]:
def filter_mag_spec ( mag_spec, rows, columns ):
    ''' Makes a filter mask specified by rows and columns. Specified rows and columns are set to 255, others 0.
    
    Parameters
    ----------
    mag_spec : 2D array
        Image with magnitude spectrum.
    Returns
    -------
    filter_mask : 2D array
        2D array mask containing 255 and 0 values.
    result : 2D array
        Visualisation of filtering.
    '''
    filter_mask = np.zeros_like(mag_spec,dtype=np.uint8)
    
    filter_mask[rows] = 255
    
    filter_mask[:,columns] = 255
        
    result = np.copy(mag_spec)
    result[filter_mask != 255] = 0
    
    return filter_mask, result

---

## Úkol

**1) Obrázek kartonu načtěte a zobrazte.**

Pozor, obrázek načtěte jako černobílý, tzn. pouze s jedním kanálem. Lze toho docílit vícero způsoby: buď můžete použít klasickou funkci `load_image` a vybrat libovolný kanál, nebo můžete použít funkci [cv2.imread](https://docs.opencv.org/3.4/d4/da8/group__imgcodecs.html#ga288b8b3da0892bd651fce07b3bbd3a56) knihovny OpenCV a předat jí parametr `cv2.IMREAD_GRAYSCALE`.

In [ ]:
cardboard_texture = ###


**2) Vyberte si z kartonu rozumně velký čtvercový výřez s nějakou vadou na kartonu. Výřez zobrazte.**

In [ ]:
cardboard_texture_cropped = ###


**3) Vyfiltrujte v rozumné míře šum mediánovým filtrem vhodné velikosti. Výsledek si pro kontrolu zobrazte.**

In [ ]:
cardboard_texture_filtered = ###


**4) Použijte min-max normalizaci k zvýšení kontrastu. Výsledek si pro kontrolu zobrazte.**

Lze použít například funkci [cv2.normalize](https://docs.opencv.org/3.4/d2/de8/group__core__array.html#ga87eef7ee3970f86906d69a92cbf064bd) knihovny OpenCV s parametry `dst=None, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U`. Parametry **alpha** a **beta** pří použití min max normalizace určují minimální a maximální cílovou hodnotu, nezapomeňte, že pracujeme s osmibitovým kanálem. 

Hint: docstring funkce lze v Jupyteru zobrazit příkazem `cv2.normalize?`.

In [ ]:
cardboard_normalized = ###


**5) Aplikujte Fourierovu transformaci na upravený obrázek. Zobrazte si spektrum obrázku.**

In [ ]:
mag_spec, fftcls_shift = ###


**6) Vyfiltrujte příslušné frekvence tak, aby obrázek obsahoval pouze neporušený karton. Vizuální výsledek si pro kontrolu zobrazte.**

Filtrace frekvencí se provádí nad získaným spektrem. Nezajímají nás vysoké frekvence (zde odpovídají vadám a šumu), ale nízké frekvence, tedy ty, které jsou blíže ke středu obrázku spektra. Pro optimální výsledek musíte experimentovat s různými hodnotami.

In [ ]:
filter_mask, filtered_spec = ###


**7) Nyní proveďte inverzní fourierovu transformaci pomocí masky získané v minulém kroku. Výsledný obrázek zobrazte.**

Výsledek inverzní Fourierovy transformace je před zobrazením třeba převést do diskrétních hodnot. Matice knihovny numpy mají metodu `astype()`, která vrátí matici zkonvertovanou do požadovaného datového typu. Pro náš osmibitový kanál je optimální použít typ **np.uint8**.

In [ ]:
img_back = ###


**8) Vytvořte rozdíl původního normalizovaného obrázku a "ideálního" obrázku, který jsme vytvořili odfiltrováním nepotřebných frekvencí. Výsledek si pro kontrolu zobrazte.**

Absolutní hodnotu rozdílu lze získat pomocí `np.abs()`. Pro zabránění přetečení se vám pro mezivýpočty bude hodit datový typ **np.int32**, výsledek nezapomeňte zkonvertovat zpět do osmibitových hodnot.

In [ ]:
diff = ###


**9) Vysegmentujte masku defektů pomocí vhodného prahování. Masku pro kontrolu zobrazte.**

In [ ]:
mask_def = ###


**10) Najděte kontury a odfiltrujte šum, výsledné kontury zakreslete do úplně původního, ale oříznutého obrázku a výsledek zobrazte.**

K nalezení kontur použijte funkci [cv2.findContours](https://docs.opencv.org/3.4/d3/dc0/group__imgproc__shape.html#ga17ed9f5d79ae97bd4c7cf18403e1689a) s parametry `mode=cv2.RETR_LIST, method=cv2.CHAIN_APPROX_SIMPLE`. K vyfiltrování miniaturních kontur se hodí funkce [cv2.contourArea](https://docs.opencv.org/3.4/d3/dc0/group__imgproc__shape.html#ga2c759ed9f497d4a618048a2f56dc97f1). Výsledek nemusí být úplně perfektní, stačí, když nalezené kontury budou přibližně na místě skutečných defektů.

In [ ]:
contours, _ = ###


...
... # filtrace kontur
...


contour_drawn = cv2.drawContours(cardboard_texture_cropped.copy(), contours, -1, color=(255, 0, 0), thickness=3)
plot_images(contour_drawn)